In [1]:
##Loading all the required libraries
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import time

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
stem = LancasterStemmer()

#library for regular expretion
import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
Fb_Data1=pd.read_csv("FB_User_Classification.csv", delimiter="\t")

In [3]:
df=Fb_Data1[['description', 'INDEX New']]
df.columns=['description', 'CATEGORY']
df['description'].replace('', np.nan, inplace=True)
df.dropna(subset=['description'], inplace=True)
df['CATEGORY'].replace('NaN', np.nan, inplace=True)
df.dropna(subset=['CATEGORY'], inplace=True)
df.head()


/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:3554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,description,CATEGORY
0,An Alladin's cave of beautiful designer brands...,No Seller
1,Everyone - let me take a minute to clarify som...,No Seller
2,CHANEL QUILTED BACKPACK SMALL 23X26CM MQH WITH...,Fake Seller
4,Longchamp Zip Around Wallet PM / Whatsapp 012-...,Reseller
5,Pre order Longchamp neo new color Pm for more ...,Reseller


In [4]:
# function to clean data
#without cleaning the accuracy is more
#try without once
stops = set(stopwords.words("english")) #Removing stop words
def cleanData(text, lowercase = False, remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z\s]',r'',txt)
    #Removing non-alpha numeric characters
    txt = re.sub(r'\n',r' ',txt)    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    
    if stemming:
        stemmer = LancasterStemmer()
        txt = " ".join([stemmer.stem(w) for w in txt.split()])

    return txt


In [5]:
df['description'] = df['description'].map(lambda x: cleanData(x, lowercase=True, remove_stops=True, stemming=True))

In [6]:
df['description']

0        An Alladin's cave of beautiful designer brands...
1        Everyone - let me take a minute to clarify som...
2        CHANEL QUILTED BACKPACK SMALL 23X26CM MQH WITH...
4        Longchamp Zip Around Wallet PM / Whatsapp 012-...
5        Pre order Longchamp neo new color Pm for more ...
6        ONHAND Longchamp Neo with strap Complete with ...
7        Practice!  aba! Go kuya, susuportahan kita pra...
8        UP FOR SWAPS/ TRADE-INS CHANEL Classic Reissue...
9        A CHI INDOVINERA' PER PRIMA/O IL NOME DI QUEST...
10       เพลงอมตะของค่าย Motown ร้องโดย โซอี เดสชาแนล ใ...
11       เพลงอมตะของค่าย Motown ร้องโดย โซอี เดสชาแนล ใ...
12       พร้อมส่งจ้า ราคา 650 บาท ส่งฟรี Ems จ้า สอบถาม...
13       Call/WhatsApp 0702216411 For your orders Deliv...
14       Call/WhatsApp 0702216411 For your orders Deliv...
15       Wenn es morgens im Bad mal wieder schell gehen...
16       Wenn es morgens im Bad mal wieder schell gehen...
17                                    Some FUN updates!!

In [7]:
#converting category column into numeric target NUM_CATEGORY column
df['NUM_CATEGORY']=df.CATEGORY.map({'No Seller':0,'Reseller':1,'Fake Seller':2})

/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [8]:
#df.drop('CATEGORY', axis=1)

In [9]:
#used in model 1,2
x=df['description']
y=df['NUM_CATEGORY']

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.85)

vect =CountVectorizer(ngram_range=(2,2))
#converting features into numeric vector
X_train = vect.fit_transform(x_train)
#converting target into numeric vector
X_test = vect.transform(x_test)


In [10]:
#Training and Predicting the data

mnb = MultinomialNB(alpha=0.2)
mnb.fit(X_train,y_train)
result= mnb.predict(X_test)

accuracy_score(result,y_test)

0.84975767366720512

In [11]:
svc = SVC(kernel = 'linear')
svc.fit(X_train,y_train)
result_svc= svc.predict(X_test)

accuracy_score(result_svc,y_test)

0.83083314101084704